The Gans Project by Balu, Marvin, Frederico and Joachim 

The first task is to get the lat., long., population, country, countrycode and wiki_dataid for a list of cities in Europe and put it in a dataframe

In [4]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
import time
!pip install python-dotenv
%load_ext dotenv
%dotenv
import os


API_key = os.getenv("API_K_C") #gets the API key from .env
#print(API_key)
#define the list of cities
cit = ['Berlin', 'London', 'Paris', 'Rome', 'Wien','Tallinn', 'Budapest', 'Warschau', 'Prague', 'Lisbon', 'Madrid', 'Amsterdam', 'Zagreb', 'Sofia', 'Athens']# use Englisch if possible
citd =['Berlin', 'Munich', 'Frankfurt', 'Kiel', 'Rostock', 'Leipzig', 'Cologne', 'Dresden', 'Erfurt', 'Jena', 'Weimar', 'Essen', 'Stuttgart', 'Mainz', 'Hannover', 'Hamburg']
#strategy is to first get the wikidataid and than collect the data from geoDB; openweather has no population data or at least I coul´nt see them 

cities_id = [] # initiate an empty id list
dfList = []
def demo(cities):
    cities_id = [] # initiate an empty id list
    dfList = []
    for city in cities:
        #retrieve the wikidataId
        time.sleep(1)
        url1 = 'https://en.wikipedia.org/wiki/{}'.format(city) #go to the wiki site of the city
        citem = requests.get(url1, 'html.parser') # get the html
        if BeautifulSoup(citem.content) != None:
            soup = BeautifulSoup(citem.content)# soup the content
        if soup.find('li', {'id':'t-wikibase'}).find('a')['href'] != None:
            wikidata_link = soup.find('li', {'id':'t-wikibase'}).find('a')['href'] #find the link that contains the wikidataid e.g. London https://www.wikidata.org/wiki/Q84
        #wl.append(wikidata_link)
        # \d+ is a regular expression and means one digit or more, the wiki data id consist of a Q followed by severaldigits
        #for group() in re see: https://www.tutorialspoint.com/What-is-the-groups-method-in-regular-expressions-in-Python
        city_id = re.search('Q\d+', wikidata_link).group()
        cities_id.append(city_id)
        #use the wikidataId to retrieve infrormation from geoDB
        url2 = "https://wft-geo-db.p.rapidapi.com/v1/geo/cities/{}".format(city_id)
        headers = {
        "X-RapidAPI-Key": API_key,
        "X-RapidAPI-Host": "wft-geo-db.p.rapidapi.com"
        }    
        response = requests.request("GET", url2, headers=headers)# gets a json-like string fromg geoDB containing the necessary informations
        print(response.json())
        cit_dic = {}#make a dictionary to retrieve the information
        cit_dic['City'] = response.json()['data']['name']
        cit_dic['Country'] = response.json()['data']['country']
        cit_dic['CountryCode'] = response.json()['data']['countryCode']
        cit_dic['WikiDataId'] = response.json()['data']['wikiDataId']
        cit_dic['Latitude'] = round(response.json()['data']['latitude'], 4)
        cit_dic['Longitude'] = round(response.json()['data']['longitude'], 4)
        cit_dic['Population'] = response.json()['data']['population']
        cit_dic['Timezone'] = response.json()['data']['timezone']
        
        dfList.append(cit_dic) #put it in a list
        df_demo = pd.DataFrame(dfList) # transform the list to df
        #print(dfList)
    return df_demo
demo(citd) #run the whole thing - good luck!

{'data': {'id': 3453309, 'wikiDataId': 'Q64', 'type': 'CITY', 'city': 'Berlin', 'name': 'Berlin', 'country': 'Germany', 'countryCode': 'DE', 'region': 'Berlin', 'regionCode': 'BE', 'elevationMeters': None, 'latitude': 52.516666666, 'longitude': 13.383333333, 'population': 3664088, 'timezone': 'Europe__Berlin', 'distance': None, 'deleted': False, 'placeType': 'CITY'}}
{'data': {'id': 3449775, 'wikiDataId': 'Q1726', 'type': 'CITY', 'city': 'Munich', 'name': 'Munich', 'country': 'Germany', 'countryCode': 'DE', 'region': 'Bavaria', 'regionCode': 'BY', 'elevationMeters': 519, 'latitude': 48.13452, 'longitude': 11.571, 'population': 1488202, 'timezone': 'Europe__Berlin', 'distance': None, 'deleted': False, 'placeType': 'CITY'}}
{'data': {'id': 3449552, 'wikiDataId': 'Q1794', 'type': 'CITY', 'city': 'Frankfurt am Main', 'name': 'Frankfurt am Main', 'country': 'Germany', 'countryCode': 'DE', 'region': 'Hesse', 'regionCode': 'HE', 'elevationMeters': 112, 'latitude': 50.113611111, 'longitude': 8

,City,Country,CountryCode,WikiDataId,Latitude,Longitude,Population,Timezone
0,Berlin,Germany,DE,Q64,52.5167,13.3833,3664088,Europe__Berlin
1,Munich,Germany,DE,Q1726,48.1345,11.5710,1488202,Europe__Berlin
2,Frankfurt am Main,Germany,DE,Q1794,50.1136,8.6797,764104,Europe__Berlin
3,Kiel,Germany,DE,Q1707,54.3233,10.1394,246601,Europe__Berlin
4,Rostock,Germany,DE,Q2861,54.0833,12.1333,209061,Europe__Berlin
5,Leipzig,Germany,DE,Q2079,51.3400,12.3750,597215,Europe__Berlin
6,Cologne,Germany,DE,Q365,50.9422,6.9578,1083498,Europe__Berlin
7,Dresden,Germany,DE,Q1731,51.0833,13.7666,554907,Europe__Berlin
8,Erfurt,Germany,DE,Q1729,50.9781,11.0292,213835,Europe__Berlin
9,Jena,Germany,DE,Q3150,50.9272,11.5864,111407,Europe__Berlin
